<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/langgraph_web_vector_db_cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langgraph langchain langchain-community chromadb sentence-transformers langchain-groq groq duckduckgo-search beautifulsoup4 requests PyPDF2 python-docx --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.7 MB/s et

In [2]:
from os.path import pathsep
import os
import requests
import tempfile
from pathlib import Path
from bs4 import BeautifulSoup
from typing import TypedDict, List, Optional

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_groq import ChatGroq
from langgraph.graph import END, StateGraph
from duckduckgo_search import DDGS
import PyPDF2
import docx
import json

In [4]:
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('groq_api_key')

In [5]:
EMBEDDING_MODEL = "all-MiniLM-L6-v2"  # Fast and efficient HuggingFace model
PERSIST_DIRECTORY = "./chroma_db"
COLLECTION_NAME = "agent_knowledge_base"

In [6]:
print("Initializing embeddings...")
embeddings=HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs={
        'device': 'cuda'
    },
    encode_kwargs={
        'normalize_embeddings': True
    }
)

Initializing embeddings...


<ipython-input-6-2614597968>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings,
    persist_directory=PERSIST_DIRECTORY
)
print("✅ Vector store initialized!")

<ipython-input-7-4019876493>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


✅ Vector store initialized!


In [8]:
def load_pdf_content(file_path):
    """Extract text from PDF file"""
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print(f"Error reading PDF: {str(e)}")
        return ""

In [9]:
def load_docx_content(file_path):
    """Extract text from DOCX file"""
    try:
        doc = docx.Document(file_path)
        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
        return text
    except Exception as e:
        print(f"Error reading DOCX: {str(e)}")
        return ""

In [10]:
def load_txt_content(file_path):
    """Extract text from TXT file"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        print(f"Error reading TXT: {str(e)}")
        return ""

In [17]:
def load_url_content(url):
    """Load content from URL"""
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove irrelevant elements
        for element in soup(["script", "style", "header", "footer", "nav", "aside"]):
            element.decompose()

        return ' '.join(soup.stripped_strings)
    except Exception as e:
        print(f"Error loading {url}: {str(e)}")
        return ""

In [18]:
def process_uploaded_file(file_path):
    """Process uploaded file based on extension"""
    file_extension = Path(file_path).suffix.lower()

    if file_extension == '.pdf':
        return load_pdf_content(file_path)
    elif file_extension == '.docx':
        return load_docx_content(file_path)
    elif file_extension in ['.txt', '.md']:
        return load_txt_content(file_path)
    else:
        print(f"Unsupported file type: {file_extension}")
        return ""


In [19]:
def add_documents_to_vectorstore(documents, source_type="unknown"):
    """Add documents to the vector store"""
    if not documents:
        print("No documents to add!")
        return

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )

    # Create Document objects with metadata
    doc_objects = []
    for i, doc_content in enumerate(documents):
        if doc_content.strip():
            chunks = text_splitter.split_text(doc_content)
            for j, chunk in enumerate(chunks):
                doc_objects.append(Document(
                    page_content=chunk,
                    metadata={
                        "source": f"{source_type}_{i}",
                        "chunk_id": j,
                        "source_type": source_type
                    }
                ))

    if doc_objects:
        vectorstore.add_documents(doc_objects)
        vectorstore.persist()
        print(f"✅ Added {len(doc_objects)} document chunks to vector store")
    else:
        print("No valid document content found!")

In [20]:
def upload_file_to_vectorstore(file_path):
    """Upload and process a file to add to vector store"""
    print(f"📁 Processing file: {file_path}")
    content = process_uploaded_file(file_path)

    if content:
        add_documents_to_vectorstore([content], f"uploaded_file_{Path(file_path).name}")
        return True
    return False

In [21]:
def load_urls_to_vectorstore(urls):
    """Load content from URLs and add to vector store"""
    print("🌐 Loading content from URLs...")
    documents = []
    for url in urls:
        content = load_url_content(url)
        if content:
            documents.append(content)

    add_documents_to_vectorstore(documents, "web_content")

In [22]:
default_urls = [
    "https://en.wikipedia.org/wiki/Large_language_model",
    "https://en.wikipedia.org/wiki/Retrieval-augmented_generation"
]

In [23]:
try:
    existing_docs = vectorstore.get()
    if not existing_docs['documents']:
        print("📚 Loading default knowledge base...")
        load_urls_to_vectorstore(default_urls)
    else:
        print(f"📚 Found {len(existing_docs['documents'])} existing documents in vector store")
except:
    print("📚 Loading default knowledge base...")
    load_urls_to_vectorstore(default_urls)

# Create retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

📚 Loading default knowledge base...
🌐 Loading content from URLs...
✅ Added 142 document chunks to vector store


<ipython-input-19-3180414562>:29: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [39]:
# Cell 8: Web Search Tool (DuckDuckGo)
def web_search(query: str, max_results: int = 3) -> str:
    """Perform web search using DuckDuckGo and return summarized results"""
    print(f"🦆 Searching DuckDuckGo for: {query}")
    try:
        with DDGS() as ddg:
            results = list(ddg.text(keywords=query, max_results=max_results))

        if not results:
            return "No web search results found."

        formatted_results = []
        for i, result in enumerate(results):
            formatted_results.append(
                f"🔗 Source {i+1}: {result.get('title', 'No title')}\n"
                f"URL: {result.get('href', 'No URL')}\n"
                f"Content: {result.get('body', 'No content')}"
            )

        return "\n\n".join(formatted_results)

    except Exception as e:
        print(f"Error in web search: {str(e)}")
        return f"Web search failed: {str(e)}"

In [40]:
print("🧠 Initializing Groq LLM...")
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",  # Fast and capable model
    temperature=0.1,
    max_tokens=1024
)

# RAG Chain
template = """Answer the question using the provided context sources.

CONTEXT:
{context}

QUESTION: {question}

INSTRUCTIONS:
- Use information from the context to answer the question
- If the context contains conflicting information, prioritize the most relevant and recent information
- If you cannot find the answer in the context, say "I don't have enough information to answer this question"
- Be concise but comprehensive in your response
- Cite sources when possible

ANSWER:"""

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

rag_chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

print("✅ RAG chain initialized!")

🧠 Initializing Groq LLM...
✅ RAG chain initialized!


In [41]:
class AgentState(TypedDict):
    question: str
    context: List[Document]
    web_context: Optional[str]
    answer: str
    next_step: str
    confidence_score: float

In [42]:
def retrieve_node(state: AgentState):
    """Retrieve documents from vector store"""
    print(f"🔍 Retrieving documents for: {state['question']}")
    try:
        context = retriever.invoke(state["question"])
        print(f"📄 Found {len(context)} relevant documents")
        return {"context": context}
    except Exception as e:
        print(f"Error in retrieval: {str(e)}")
        return {"context": []}


In [43]:
def web_search_node(state: AgentState):
    """Perform web search for additional information"""
    web_results = web_search(state["question"])
    return {"web_context": web_results, "next_step": "generate"}

def generate_node(state: AgentState):
    """Generate final answer using available context"""
    print("💡 Generating answer...")

    # Prepare local context
    if state["context"]:
        local_context = "\n\n".join([
            f"📄 Document {i+1} (Source: {doc.metadata.get('source', 'unknown')}):\n{doc.page_content}"
            for i, doc in enumerate(state["context"])
        ])
    else:
        local_context = "No relevant documents found in local knowledge base."

    # Combine contexts
    if state.get("web_context"):
        full_context = f"LOCAL KNOWLEDGE BASE:\n{local_context}\n\nWEB SEARCH RESULTS:\n{state['web_context']}"
    else:
        full_context = f"LOCAL KNOWLEDGE BASE:\n{local_context}"

    # Generate answer
    try:
        result = rag_chain.invoke({
            "question": state["question"],
            "context": full_context
        })
        return {"answer": result}
    except Exception as e:
        return {"answer": f"Error generating answer: {str(e)}"}


In [44]:
def decision_node(state: AgentState):
    """Decide whether to use web search based on context relevance"""
    if not state["context"]:
        print("🤔 No local context found, using web search")
        return {"next_step": "web_search", "confidence_score": 0.0}

    # Simple relevance scoring based on context length and keyword matching
    context_text = " ".join([doc.page_content for doc in state["context"]]).lower()
    question_keywords = state["question"].lower().split()

    # Calculate simple relevance score
    keyword_matches = sum(1 for keyword in question_keywords if keyword in context_text)
    relevance_score = keyword_matches / len(question_keywords) if question_keywords else 0

    # Also consider context length (more content might be more comprehensive)
    context_length_score = min(len(context_text) / 1000, 1.0)  # Normalize to 0-1

    confidence_score = (relevance_score + context_length_score) / 2

    print(f"📊 Context confidence score: {confidence_score:.2f}")

    # Use web search if confidence is low
    if confidence_score < 0.3:
        print("🌐 Low confidence in local context, adding web search")
        return {"next_step": "web_search", "confidence_score": confidence_score}
    else:
        print("✅ Sufficient local context found")
        return {"next_step": "generate", "confidence_score": confidence_score}

# Cell 12: Build Agent Workflow
print("🤖 Building agent workflow...")

workflow = StateGraph(AgentState)

🤖 Building agent workflow...


In [45]:
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("decision", decision_node)
workflow.add_node("web_search", web_search_node)
workflow.add_node("generate", generate_node)

# Set entry point and edges
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "decision")
workflow.add_conditional_edges(
    "decision",
    lambda state: state["next_step"],
    {
        "web_search": "web_search",
        "generate": "generate"
    }
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

# Compile the agent
agent = workflow.compile()
print("✅ Agent workflow compiled successfully!")

✅ Agent workflow compiled successfully!


In [46]:
def ask_question(question: str, show_context: bool = False):
    """Run the agent with a question"""
    print(f"\n" + "="*60)
    print(f"❓ Question: {question}")
    print("="*60)

    initial_state = {
        "question": question,
        "context": [],
        "web_context": None,
        "answer": "",
        "next_step": "retrieve",
        "confidence_score": 0.0
    }

    try:
        result = agent.invoke(initial_state)

        print(f"\n✅ Answer:")
        print(f"{result['answer']}")

        if show_context and result.get('context'):
            print(f"\n📚 Retrieved Context:")
            for i, doc in enumerate(result['context'][:2]):  # Show first 2 docs
                print(f"  {i+1}. {doc.page_content[:200]}...")

        if result.get('confidence_score') is not None:
            print(f"\n📊 Confidence Score: {result['confidence_score']:.2f}")

        return result

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None


In [47]:
def upload_and_ask(file_path: str, question: str):
    """Upload a file and then ask a question"""
    print(f"📁 Uploading file: {file_path}")

    if upload_file_to_vectorstore(file_path):
        print("✅ File uploaded successfully!")
        # Recreate retriever to include new documents
        global retriever
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}
        )
        return ask_question(question)
    else:
        print("❌ Failed to upload file")
        return None

In [48]:
print("\n🚀 Testing the Enhanced Agent System")
print("="*50)


🚀 Testing the Enhanced Agent System


In [49]:
# Test with local knowledge
ask_question("What is a large language model?")

# Test with a question that might need web search
ask_question("What are the latest developments in AI for 2025?")

# Test with a technical question
ask_question("How does retrieval-augmented generation work?")


❓ Question: What is a large language model?
🔍 Retrieving documents for: What is a large language model?
📄 Found 3 relevant documents
📊 Context confidence score: 1.00
✅ Sufficient local context found
💡 Generating answer...

✅ Answer:
A large language model refers to a type of artificial intelligence (AI) designed to process and understand human language. These models have been developed over the years, with significant advancements in recent times. Before 2017, there were a few language models that were considered large compared to the capacities available at the time (Document 3). The development of large language models has been influenced by the use of statistical language modeling, neural networks, and the availability of large datasets (Document 3).

According to Blank (2023), large language models are supposed to model language capacity, but their abilities are still being debated (Document 1). A survey on multimodal large language models highlights the complexities and challenge

{'question': 'How does retrieval-augmented generation work?',
 'context': [Document(metadata={'chunk_id': 4, 'source_type': 'web_content', 'source': 'web_content_1'}, page_content='] Retrieval-Augmented Generation (RAG) enhances large language models (LLMs) by incorporating an information-retrieval mechanism that allows models to access and utilize additional data beyond their original training set. AWS states, "RAG allows LLMs to retrieve relevant information from external data sources to generate more accurate and contextually relevant responses" ( indexing ). [ 10 ] This approach reduces reliance on static datasets, which can quickly become outdated. When a user submits a query, RAG uses a document retriever to search for relevant content from available sources before incorporating the retrieved information into the model’s response ( retrieval ). [ 11 ] Ars Technica notes that "when new information becomes available, rather than having to retrain the model, all that’s needed is to 

In [50]:
# Cell 15: File Upload Example (Uncomment and modify as needed)
file_path = "/content/PraveenResume.pdf"  # Replace with actual file path
question = "What is the main topic discussed in this document?"
upload_and_ask(file_path, question)

print("\n📝 Usage Instructions:")
print("1. To upload a file: upload_and_ask('path/to/file.pdf', 'your question')")
print("2. To ask questions: ask_question('your question')")
print("3. Supported file types: PDF, DOCX, TXT, MD")
print("4. The system combines local knowledge with web search when needed")


📁 Uploading file: /content/PraveenResume.pdf
📁 Processing file: /content/PraveenResume.pdf
✅ Added 4 document chunks to vector store
✅ File uploaded successfully!

❓ Question: What is the main topic discussed in this document?
🔍 Retrieving documents for: What is the main topic discussed in this document?
📄 Found 3 relevant documents
📊 Context confidence score: 0.78
✅ Sufficient local context found
💡 Generating answer...

✅ Answer:
The main topic discussed in this document is Large Language Models, specifically their development, training, and applications in natural language processing. This is evident from the content of Document 2, which explicitly mentions "Large language model" and categories such as "Artificial neural networks", "Machine learning", and "Natural language processing" (Source: web_content_0). Additionally, Document 3 provides further information on related topics, including prompt design, token healing, and deduplicating training data for language models (Source: web

In [51]:
# Cell 16: Additional Utility Functions
def show_vector_store_stats():
    """Display statistics about the vector store"""
    try:
        collection = vectorstore.get()
        total_docs = len(collection['documents'])

        # Count by source type
        source_types = {}
        for metadata in collection['metadatas']:
            source_type = metadata.get('source_type', 'unknown')
            source_types[source_type] = source_types.get(source_type, 0) + 1

        print(f"\n📊 Vector Store Statistics:")
        print(f"Total documents: {total_docs}")
        print(f"Source breakdown:")
        for source_type, count in source_types.items():
            print(f"  - {source_type}: {count} chunks")

    except Exception as e:
        print(f"Error getting stats: {str(e)}")

def clear_vector_store():
    """Clear all documents from the vector store"""
    try:
        vectorstore.delete_collection()
        print("🗑️ Vector store cleared successfully!")
        # Reinitialize
        vectorstore, retriever
        vectorstore = Chroma(
            collection_name=COLLECTION_NAME,
            embedding_function=embeddings,
            persist_directory=PERSIST_DIRECTORY
        )
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    except Exception as e:
        print(f"Error clearing vector store: {str(e)}")

# Show current stats
show_vector_store_stats()

print("\n🎉 Enhanced Agentic Workflow Ready!")
print("📋 Available functions:")
print("  - ask_question(question, show_context=False)")
print("  - upload_and_ask(file_path, question)")
print("  - show_vector_store_stats()")
print("  - clear_vector_store()")


📊 Vector Store Statistics:
Total documents: 150
Source breakdown:
  - web_content: 142 chunks
  - uploaded_file_PraveenResume.pdf: 8 chunks

🎉 Enhanced Agentic Workflow Ready!
📋 Available functions:
  - ask_question(question, show_context=False)
  - upload_and_ask(file_path, question)
  - show_vector_store_stats()
  - clear_vector_store()
